# Advanced Filters

In this module, we will take a closer look at common filtering patterns.  Note that this list is based on the Common Filter Operations section of the [SQL Alchemy tutorial](https://docs.sqlalchemy.org/en/latest/orm/tutorial.html) from the SQL Alchemy documentation, which is copyright © by SQLAlchemy authors and contributors. SQLAlchemy and its documentation are licensed under the MIT license.

In [1]:
import pandas as pd
from dfply import *
import seaborn as sns
%matplotlib inline

### Common Filter Operators

Most filters consist of the following operations.

* Equals/not equals and other inequalities
* Like/ilike
* In/not in
* Is Null/is not null
* And/or


## Inequalities

In this (short) lecture, we will review filtering based on inequalities.

In [2]:
from more_dfply import fix_names
heroes_raw = pd.read_csv('./data/heroes_information.csv', na_values=['-', '-99.0', ''])
heroes = (heroes_raw >> fix_names)
heroes.head()

,Unnamed_0,name,Gender,Eye_color,Race,Hair_color,Height,Publisher,Skin_color,Alignment,Weight
0,0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,NaN,good,441.0
1,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
3,3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,NaN,bad,441.0
4,4,Abraxas,Male,blue,Cosmic Entity,Black,NaN,Marvel Comics,NaN,bad,NaN


## Category 1 - Equality and Inequality

In all three frameworks, equalities/inequalities are performed using the regular Python operators on column expressions.

#### equals:

In [3]:
(heroes
 >> filter_by(X.Eye_color == 'blue')
 >> head(2))

,Unnamed_0,name,Gender,Eye_color,Race,Hair_color,Height,Publisher,Skin_color,Alignment,Weight
1,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0


In [4]:
(heroes
 >> filter_by(X.Eye_color.eq('blue'))
 >> head(2))

,Unnamed_0,name,Gender,Eye_color,Race,Hair_color,Height,Publisher,Skin_color,Alignment,Weight
1,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0


#### not equals:

In [5]:
(heroes
 >> filter_by(X.Eye_color != 'blue')
 >> head(2))

,Unnamed_0,name,Gender,Eye_color,Race,Hair_color,Height,Publisher,Skin_color,Alignment,Weight
0,0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,NaN,good,441.0
3,3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,NaN,bad,441.0


In [6]:
(heroes
 >> filter_by(~X.Eye_color.eq('blue'))
 >> head(2))

,Unnamed_0,name,Gender,Eye_color,Race,Hair_color,Height,Publisher,Skin_color,Alignment,Weight
0,0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,NaN,good,441.0
3,3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,NaN,bad,441.0


#### Other inequalities

In [7]:
(heroes
 >> filter_by(X.Height > 200)
 >> filter_by(X.Weight <= 440)
 >> head(2))

,Unnamed_0,name,Gender,Eye_color,Race,Hair_color,Height,Publisher,Skin_color,Alignment,Weight
17,17,Alien,Male,NaN,Xenomorph XX121,No Hair,244.0,Dark Horse Comics,black,bad,169.0
19,19,Amazo,Male,red,Android,NaN,257.0,DC Comics,NaN,bad,173.0


## Reminder - Referencing Constructed Column

Recall that we can reference a constructed column by using `X` in `pandas` + `dfply`

In [8]:
(heroes
 >> mutate(Weight_kg = X.Weight/2.2046)
 >> filter_by(X.Weight_kg <= 200)
 >> head(2))

,Unnamed_0,name,Gender,Eye_color,Race,Hair_color,Height,Publisher,Skin_color,Alignment,Weight,Weight_kg
1,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0,29.483807
2,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0,40.823732


## <font color="red"> Exercise 4.1.1 - The Super Hero Dating Game - Part 1</font>

Yesterday, you notice a singles add in the local paper that reads

> SWF looking for BESHM (blue-eyed super hero).  Must be tall (70+ inches).  Only interested in bad boys! Must list height (in inches) in reply!

Write a query in each framework to help find candidates for this personal add.

In [12]:
# Your solution here
(heroes
 >> mutate(Height_inch = X.Height/2.54)
 >> filter_by(X.Height_inch > 70,
             X.Gender == 'Male',
             X.Eye_color.eq('blue'),
             X.Alignment == 'bad')
 >> drop(X.Height)
 >> head())

,Unnamed_0,name,Gender,Eye_color,Race,Hair_color,Publisher,Skin_color,Alignment,Weight,Height_inch
5,5,Absorbing Man,Male,blue,Human,No Hair,Marvel Comics,NaN,bad,122.0,75.984252
11,11,Air-Walker,Male,blue,NaN,White,Marvel Comics,NaN,bad,108.0,74.015748
48,48,Atlas,Male,blue,God / Eternal,Brown,DC Comics,NaN,bad,126.0,77.952756
109,109,Blackwing,Male,blue,NaN,Black,Marvel Comics,NaN,bad,86.0,72.834646
140,140,Bullseye,Male,blue,Human,blond,Marvel Comics,NaN,bad,90.0,72.047244
